# Parameters

In [ ]:
NUM_EPOCHS = 30
BOW = "tf_idf"
NETWORK = "ffnn"
NROWS = 100

# RNN
HIDDEN_SIZE = 100
NUM_LAYERS = 1

LEARNING_RATE = 0.001

BATCH_SIZE = 50

LOAD_PATH = None
SAVE_PATH = "OverwriteMe.pt"

## Import the necessary functions and libraries

In [1]:
from BoW import TF_IDF
from Preprocessing import getDataFrameFromData
from rnn import RNN
from networkFunctions import train, test
from ffnn import FFNN
from torch.utils.data import DataLoader
from dataSet import CustomDataDataSet

import torch 
import torch.nn as nn

## Read Data

In [5]:
train_df, train_labels = getDataFrameFromData("Archive/arxiv_train.csv", nrows=NROWS)
test_df, test_labels = getDataFrameFromData("Archive/arxiv_test.csv", nrows=NROWS)

,id,abstract,label
0,31716,automatic meeting analysis is an essential f...,eess
1,89533,we propose a protocol to encode classical bi...,quantph
2,82700,a number of physically intuitive results for...,quantph
3,78830,in the last decade rareearth hexaborides hav...,physics
4,94948,we introduce the weak barycenter of a family...,stat
...,...,...,...
95,36235,in this paper a new framework for optimizing...,eess
96,38205,reconfigurable intelligent surface ris techn...,eess
97,55889,we explore emphfolded spinning string config...,hepth
98,32525,in this paper we formulate a simple algorith...,eess


## Calculate BoW and create dataloader

In [7]:
if BOW == "tf_idf":
    train_tensors = TF_IDF(train_df)
    test_tensors = TF_IDF(test_df)
elif BOW == "countVector":
    pass
else:
    raise SyntaxError(f"Ugyldig bow: {BOW}")


train_dataset = CustomDataDataSet(train_tensors, train_labels)
test_dataset = CustomDataDataSet(test_tensors, test_labels)
train_dataloader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=True)

c:\UIS\DAT550\dat550-prosjekt\BoW.py:26: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:620.)
  tensor = torch.sparse.FloatTensor(i, v, torch.Size(shape))


tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

## Create Network Model

In [ ]:
if NETWORK == "ffnn":
    model = FFNN(train_tensors.shape[1])
elif NETWORK == "rnn":
    model = RNN(train_tensors.shape[1], HIDDEN_SIZE, NUM_LAYERS)

## Load state dict if desired

In [ ]:
if LOAD_PATH is not None:
	try:
		model.load_state_dict(torch.load(LOAD_PATH))
		print("Loaded model state")
	except FileNotFoundError:
		raise SyntaxError(f"Fant ingen fil: {LOAD_PATH}")

# Run testing. Train and save parameters if not pre-trained

In [ ]:
loss_function = nn.CrossEntropyLoss()

if LOAD_PATH is None:
    optimiser = torch.optim.adam(model.parameters(), lr=LEARNING_RATE)

    for i in range(NUM_EPOCHS):
        train(train_dataloader, model, optimiser, loss_function)
        test(test_dataloader, model, loss_function)
    
    torch.save(model.state_dict(), SAVE_PATH)

else:
    for i in range(NUM_EPOCHS):
        test(test_dataloader, model, loss_function)